In [ ]:

import pickle
from speclearn.tools.data_tools import *
from speclearn.plot.map import *
from speclearn.io.data.aoi import get_full_map_aoi
from speclearn.deep_learning.predict import get_data, predict_full_map, get_full_data, get_latent
from speclearn.deep_learning.model_utils import load_beta_VAE_model, get_colorbar
from speclearn.deep_learning.ml_tools import (get_poles_aoi)
from sklearn.cluster import KMeans
from matplotlib.text import Text
from speclearn.deep_learning.cluster import cluster_with_kmeans
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from speclearn.io.transform.bin_tools import make_bins
from speclearn.deep_learning.model_utils import make_model, train_model
from speclearn.io.data.aoi import get_aoi_list_with_full_path, get_full_map_aoi_longitude
import wandb
local_wavelength = select_wavelength(s_0=0, s_1=-12)
import datetime
print('Current time: ', datetime.datetime.now())
figure_dir = '/home/freya/Documents/figures'
from speclearn.tools.constants import *
import matplotlib
from speclearn.deep_learning.ml_tools import Config, divide_data

from datetime import datetime
import torch
now = datetime.now() # current date and time
date_time = now.strftime("%m%d%Y_%H%M%S")
print("date and time:",date_time)
date_time[0]


def get_colorbar_1(k):
    map = 'viridis'
    n_colors=200

    palette = sns.color_palette(map, n_colors=n_colors)
    b=[sns.color_palette("Set2")[-1]]

    if k == 8:
        a= [palette[0], palette[30],palette[60], palette[90]]
    elif k == 6:
        a= [palette[0], palette[60]]
    elif k == 5:

        a= [palette[30] ,palette[100], b[0], palette[160], palette[199]]    
    else:
        a = [palette[30],palette[60], palette[90]]

    c= [palette[120],palette[160], palette[199]]

    clist = a[::-1]+b+c[::-1]
    if k == 2:
        clist = [clist[0]]+[clist[-2]]
    if k == 5:
        
        a = ['#432e6b','#4580ba', '#b3b3b3', '#7cd250', '#fbeb37'] #fced69
        clist = a


    cmap = matplotlib.colors.ListedColormap(clist, "")
    cmap.set_under('black')

    return clist, cmap

def make_full_cluster_map_1(area='large',save_file=False):
    cluster_2d_s = cluster_2d[:, 0:600]
    #cluster_2d_s = np.fliplr(cluster_2d_s)
    
    cluster_2d_n = cluster_2d[:, -600:]
    clist, cmap = get_colorbar_1(5)
    if area == 'large':
        d_area = 30
    else:
        d_area = 10
    lat_edges, lat_centers = make_bins(aoi_s['lat_range'], int(
        d_area/aoi_s['d_lat']))
    long_edges, long_centers = make_bins(
        aoi_s['long_range'], int(360/aoi_s['d_long']))
    loc_s = {'long_center': long_centers,
            'lat_center': lat_centers, }
    df_2d_s = get_df_2d(np.array(cluster_2d_s), loc_s)
    r_s, theta_s = get_polar_loc(df_2d_s, 'south')

    lat_edges, lat_centers = make_bins(aoi_n['lat_range'], int(d_area/aoi_n['d_lat']))
    long_edges, long_centers = make_bins(
        aoi_n['long_range'], int(360/aoi_n['d_long']))
    loc_n = {'long_center': long_centers,
            'lat_center': lat_centers, }
    df_2d_n = get_df_2d(np.array(cluster_2d_n), loc_n)
    r_n, theta_n = get_polar_loc(df_2d_n, 'north')

    fig = plt.figure(figsize=(15, 10))
    ax1 = fig.add_axes([0.0, 0.48, 0.8, 0.5])
    ax2 = fig.add_axes([0.0, 0.0, 0.45, 0.4], polar=True)
    ax3 = fig.add_axes([0.4, 0.0, 0.45, 0.4], polar=True)

    cbar_ax = fig.add_axes([0.85, 0.1, 0.015, 0.9])  # left, bottom, width, height
    cbar_ax.set_yticks(np.linspace(0.5, k-0.5, k))
    cbar_ax.set_yticklabels(np.linspace(1, k, k, dtype=int))    
    fig.subplots_adjust(right=0.8)
    clist, cmap = get_colorbar_1(k)
    sns.heatmap(np.rot90(cluster_2d[:,400:-400]), cmap=cmap, ax=ax1,#+0.5
                cbar_ax=cbar_ax, vmin=0, vmax=k, cbar_kws={'label': 'Cluster', 'ticks': np.linspace(0.5, k-0.5, k), 'boundaries': np.arange(k+1), 'extend': 'neither', 'extendfrac': 'auto'})

    ax1.set_xticks(np.linspace(0.5, np.rot90(cluster_2d[:,400:-400]).shape[1]-1.0, 19))
    ax1.set_xticklabels(f'{c:.1f}' for c in np.linspace(-180, 180, 19, dtype=int))
    ax1.set_yticks(np.linspace(0.5, np.rot90(cluster_2d[:,400:-400]).shape[0]-1.0, 15))
    ax1.set_yticklabels(f'{c:.1f}' for c in np.flip(
        np.linspace(-70, 70, 15, dtype=int)))
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')

    dr = np.fabs((r_s[0] - r_s[1]) / 2)
    ax2.set_rlim(np.min(r_s) - dr, np.max(r_s) + dr)
    ax2.set_theta_direction(1)
    ax2.set_theta_offset(np.pi)
    ax2.pcolormesh(theta_s, r_s, df_2d_s, cmap=cmap, vmin=0, vmax=k)
    ax2.set_yticklabels([])
    ax2.set_theta_zero_location("N")
    ax2.set_theta_direction(-1) 

    ax2.set_xticklabels([Text(0.0, 0, '0°'),
                        Text(0.7853981633974483, 0, '45°'),
                        Text(1.5707963267948966, 0, '90°'),
                        Text(2.356194490192345, 0, '135°'),
                        Text(3.141592653589793, 0, '180°'),
                        Text(3.9269908169872414, 0, '-135°'),
                        Text(4.71238898038469, 0, '-90°'),
                        Text(5.497787143782138, 0, '-45°')])

    dr = np.fabs((r_n[0] - r_n[1]) / 2)
    ax3.set_rlim(np.min(r_n) - dr, np.max(r_n) + dr)
    ax3.set_theta_direction(1)
    ax3.set_theta_offset(np.pi / 2.0)
    ax3.set_theta_zero_location("S")
    ax3.set_xticklabels([Text(0.0, 0, '0°'),
                        Text(0.7853981633974483, 0, '45°'),
                        Text(1.5707963267948966, 0, '90°'),
                        Text(2.356194490192345, 0, '135°'),
                        Text(3.141592653589793, 0, '180°'),
                        Text(3.9269908169872414, 0, '-135°'),
                        Text(4.71238898038469, 0, '-90°'),
                        Text(5.497787143782138, 0, '-45°')])
    ax3.pcolormesh(theta_n, r_n, df_2d_n, vmin=0, vmax=k, cmap=cmap)
    ax3.set_yticklabels([])
    cbar_ax.set_yticks(np.linspace(0.5, k-0.5, k))
    cbar_ax.set_yticklabels(np.linspace(1, k, k, dtype=int))
    ax2.grid(False)
    ax3.grid(False)
    #if full:
    if save_file:
        plt.savefig(f'{FIGURE_DIR}/results/{date_time}_map_clusters_{k}_{area}_{model_name}_full.png', bbox_inches='tight', dpi=1600)
        print(f'{FIGURE_DIR}/results/{date_time}_map_clusters_{k}_{area}_{model_name}_full.png')
        #plt.savefig(f'{figure_dir}/cluster_maps/{date_time}_map_clusters_{k}_{area}_{model_name}_full.pdf', bbox_inches='tight')
    #else:
        #plt.savefig(f'{figure_dir}/cluster_maps/{date_time}_map_clusters_{k}_{area}_{model_name}.png', #bbox_inches='tight', dpi=300)
        #plt.savefig(f'{figure_dir}/cluster_maps/{date_time}_map_clusters_{k}_{area}_{model_name}.pdf', #bbox_inches='tight')
    plt.show()

def swap_colors(cluster_2d, color_1, color_2):
    if color_2 == 0:
        color_2 = color_1
        color_1 = 0

    cluster_2d[cluster_2d == color_1] = -color_1
    cluster_2d[cluster_2d == color_2] = -color_2
    cluster_2d[cluster_2d == -color_1] = color_2
    cluster_2d[cluster_2d == -color_2] = color_1

    return

# Settings

In [2]:
k = 5
crs = False
area = 'large'
norm = False
full = False
clist, cmap = get_colorbar(k)

# Preparation

In [ ]:
#cmap = get_colorbar(k)
aoi_list = get_full_map_aoi_longitude(step_size=20)
#model, model_name = load_beta_VAE_model(crs=crs, norm=norm)

no_latent = 5
config = {
    "model": "CVAE",
    "learning_rate": 0.00001,
    "epochs": 100,
    "no_batches": 256,
    "no_latent": no_latent,
    "type": 'Adam',
    "loss_function": 'MSE',
    "input_size": 71,
    "beta": 1,
    "patience": 5,
    "architecture": 2,
    "activation": 'sigmoid',
    "dropout": 0.5,
    "architecture": 4
}
run_name='a8h7e21r'
model_name=run_name

model, criterion, optimizer = make_model(Config(config))

prev_model = wandb.restore(
    f'{run_name}_model.h5', f'freja-thoresen/M3-autoencoders/{run_name}', replace=True) 

model.load_state_dict(torch.load(prev_model.name,map_location=torch.device('cpu')))

print('model:', model_name)

## Predict with model

In [ ]:
predict_full_map(aoi_list, model, model_name, crs=crs, norm=norm, no_latent=6)

In [8]:
#data_2d, coord, latent, recon = get_data(aoi_list, model_name, crs=crs, norm=norm)
#recon = None # Release from memory
from speclearn.deep_learning.predict import get_coord

data_2d = get_full_data(aoi_list, crs, norm, periods=[])
coord = get_coord(aoi_list)
latent = get_latent(aoi_list, model_name)

## Cluster data

In [ ]:
kmeans_name = f"kmeans_{k}_{model_name}.pkl"
kmeans = cluster_with_kmeans(kmeans_name, data_2d[:, 200:-200,:], latent, k, replace=True)

In [10]:
cluster, cluster_2d = predict_kmeans(kmeans, data_2d, latent, coord)

## Prepare poles data

In [11]:
aoi_s = get_poles_aoi(area=area)
aoi_n = get_poles_aoi(area=area, region='north')

# Eq. and poles map

In [ ]:
clist, cmap = get_colorbar_1(k)

In [ ]:
make_full_cluster_map_1(save_file=True)

In [15]:
# swap_colors(cluster_2d, 1, 3)

In [92]:
# np.save(f'{CACHE_CLUSTER}/{model_name}_{k}_cluster_2d.npy', cluster_2d)
#np.save(f'{CACHE_CLUSTER}/{model_name}_{k}_cluster_2d_50_lat_south.npy', cluster_2d[:, 0:600])
#np.save(f'{CACHE_CLUSTER}/{model_name}_{k}_cluster_2d_50_lat_north.npy',cluster_2d[:, -600:])